In [1]:
import sqlite3
import pandas as  pd
import numpy as np
import datetime
import os

### <span style="color:#9932CC">**1. Diretórios Mapeados**</span>

In [2]:
result_file = '../../Data/Operacao/dados_result.parquet'

path_database = '../../Code/Operationalization/db.sqlite3'

### <span style="color:#9932CC">**2. Leitura dos dados tratados**</span>

In [3]:
dataframe = pd.read_parquet(result_file)


In [5]:
dataframe.head(2)

,id,dsc_texto,tip_supervisao,din_execucao,tip_predicao,pct_predicao
0,832897,desligamento involuntario da unidade geradora ...,oper,2020-09-12 21:39:23.492051,True,78.2
1,832899,sera realizado manutencao corretiva no transfo...,oper,2020-09-12 21:39:23.492051,True,97.3


### <span style="color:#9932CC">**3. Inclusão ou Atualização na base de dados**</span>

In [ ]:
sqliteConnection = sqlite3.connect(path_database)
cursor = sqliteConnection.cursor()
    
for index,row in dataframe.iterrows():
    df_id  = dataframe.loc[index,'id']
    df_dsc_texto  = dataframe.loc[index,'dsc_texto']
    df_tip_supervisao  = dataframe.loc[index,'tip_supervisao']
    df_din_execucao  = dataframe.loc[index,'din_execucao']
    df_tip_predicao  = dataframe.loc[index,'tip_predicao']
    df_pct_predicao  = dataframe.loc[index,'pct_predicao']
    
    array_id = np.array([df_id]) 
    array_insert = np.array([df_id,df_dsc_texto,df_tip_supervisao,df_din_execucao,df_tip_predicao,df_pct_predicao]) 
    array_update = np.array([df_din_execucao, df_tip_supervisao, df_tip_predicao, df_pct_predicao, df_id]) 
       
    
    try:
        
        #print("Conectado com sucesso ao SQLite")
        sqlite_query = """ SELECT COUNT(id) as total FROM applications_predicao WHERE id = ? """
        cursor.execute(sqlite_query, array_id)
        (total,) = cursor.fetchone()
        
        if total == 0 : 
            sqlite_insert_query = """INSERT INTO applications_predicao(id,dsc_texto,tip_supervisao,din_execucao,tip_predicao,pct_predicao)
                                     VALUES (?,?,?,?,?,?)"""
            cursor.execute(sqlite_insert_query,array_insert)
            sqliteConnection.commit()
            #print('')
            # print("Total=", cursor.rowcount, " registros inseridos com sucesso")   
        
        else:
            sqlite_update_query = """UPDATE applications_predicao SET din_execucao =  ?, tip_supervisao = ? , tip_predicao = ? ,
                                     pct_predicao = ?  WHERE id = ?  """
            cursor.execute(sqlite_update_query, array_update)
            #sqliteConnection.commit()
           # print('')
           # print("Total=", cursor.rowcount, "registros atualizados com sucesso")     
                        
    except sqlite3.Error as error:
        print("erro = ", error)
    
   

cursor.close()
sqliteConnection.close()
print("Inclusao e Atualização realizadas com sucesso")
print("Conexão com SQLite fechada")
print()
      

### <span style="color:#9932CC">**5. Formatação Percentual**</span>

In [8]:
sqliteConnection = sqlite3.connect(path_database)
cursor = sqliteConnection.cursor()

try:    
    sqlite_update_query = """UPDATE applications_predicao set pct_predicao = CAST(CAST(pct_predicao AS DECIMAL(4,2)) AS VARCHAR(10))"""
    cursor.execute(sqlite_update_query)
    sqliteConnection.commit()
    cursor.close()    
   

except sqlite3.Error as error:
    print("erro = ", error)
    
    
finally:
   if (sqliteConnection):
        sqliteConnection.close() 

### <span style="color:#9932CC">**6. Atualiza Estatistica**</span>

In [9]:
df_data = pd.DataFrame(data=dataframe.din_execucao.astype(str))
df_data = df_data.min()

array_df_data = df_data.to_numpy()
#array_df_data


sqliteConnection = sqlite3.connect(path_database)

cursor = sqliteConnection.cursor()
print("Conectado com sucesso ao SQLite")

try:
    
    # BLOCO - TRUE
    # ---------------------------------
    sqlite_select_true_query = """SELECT COUNT(*) as total_true from applications_predicao 
                                WHERE tip_predicao = 'True' and din_execucao = ? """
 
    cursor.execute(sqlite_select_true_query, array_df_data)
    (total_true,) = cursor.fetchone()
    a = np.array(total_true)
    array_true = np.append(array_df_data, a)  
       
    sqlite_insert_queryTrue = """INSERT INTO applications_estatistica(din_execucao, dsc_predicao, qtd_registro) 
                             VALUES  (?, 'True', ? ) """
    
    cursor.execute(sqlite_insert_queryTrue, array_true)
    sqliteConnection.commit()
    
    
    
     # BLOCO -  FALSE
     # ---------------------------------
    sqlite_select_false_query = """SELECT COUNT(*) as total_false from applications_predicao 
                          WHERE tip_predicao = 'False' and din_execucao = ? """
 
    cursor.execute(sqlite_select_false_query, array_df_data)
    (total_false,) = cursor.fetchone()
    b = np.array(total_false)
    array_false = np.append(array_df_data, b)  

    sqlite_insert_queryFalse = """INSERT INTO applications_estatistica(din_execucao, dsc_predicao, qtd_registro) 
                             VALUES  (?, 'False', ? ) """
    
    cursor.execute(sqlite_insert_queryFalse, array_false)
    sqliteConnection.commit()
    
       
    # Final do Processo
    cursor.close()

except sqlite3.Error as error:
    print("Falha em INSERT no database", error) 
  
    
finally:
   if (sqliteConnection):
        sqliteConnection.close()  
        print("Estatistica Finalizada")  
        print("")
        print("")               

Conectado com sucesso ao SQLite
Estatistica Finalizada


